# Spaceship Titanic GRANDE

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']      = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import pandas as pd
from sklearn.model_selection import train_test_split
from GRANDE import GRANDE
import optuna
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

SEED = 42

## Fitting the GRANDE model

The code is from the [GRANDE github repository](https://github.com/s-marton/GRANDE).

In [2]:
# 1. Load & preprocess
df = pd.read_csv('data/train_processed.csv')
test_df = pd.read_csv('data/test_processed.csv')

In [3]:
print(df.dtypes)

PassengerId                   object
Age                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Transported                     bool
CabinNum                       int64
HomePlanet_Europa               bool
HomePlanet_Mars                 bool
CryoSleep_True                  bool
Destination_PSO J318.5-22       bool
Destination_TRAPPIST-1e         bool
VIP_True                        bool
Deck_B                          bool
Deck_C                          bool
Deck_D                          bool
Deck_E                          bool
Deck_F                          bool
Deck_G                          bool
Deck_T                          bool
Side_S                          bool
dtype: object


In [4]:
print(test_df.dtypes)

PassengerId                   object
Age                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
CabinNum                       int64
HomePlanet_Europa               bool
HomePlanet_Mars                 bool
CryoSleep_True                  bool
Destination_PSO J318.5-22       bool
Destination_TRAPPIST-1e         bool
VIP_True                        bool
Deck_B                          bool
Deck_C                          bool
Deck_D                          bool
Deck_E                          bool
Deck_F                          bool
Deck_G                          bool
Deck_T                          bool
Side_S                          bool
dtype: object


In [ ]:
X = df.drop(columns=['PassengerId','Transported']).values
y = df['Transported'].astype(int).values
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=SEED,
    stratify=y
)

categorical_feature_indices = []

args = {
    'epochs': 100,
    'early_stopping_epochs': 10,
    'batch_size': 64,
    'cat_idx': categorical_feature_indices,
    'objective': 'binary',
    'random_seed': SEED,
    'verbose': 1,
}

# 3) make one “sampler” GRANDE (params can be empty dict)
sampler = GRANDE(params={}, args=args)

def objective(trial):
    tp = sampler.define_trial_parameters(trial, args)
    model = GRANDE(params=tp, args=args)
    model.fit(
        X_train=X_train,
        y_train=y_train,
        X_val=X_valid,
        y_val=y_valid
    )
    preds = model.predict(X_valid)          # shape = (n_samples, 2)
    y_pred = (preds[:,1] > 0.5).astype(int)
    return accuracy_score(y_valid, y_pred)

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=20)

print("Best CV accuracy:", study.best_value)
print("Best params:", study.best_params)

# 5) finally train your “production” model
best_params = study.best_params
final_model = GRANDE(params=best_params, args=args)
final_model.fit(
    X_train=X_train,
    y_train=y_train,
    X_val=X_valid,
    y_val=y_valid
)

[I 2025-04-27 17:14:05,144] A new study created in memory with name: no-name-e2daa48c-0a49-4f46-a220-5ba045b05abb


Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - loss: 0.7228 - val_loss: 0.6305
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - loss: 0.5946 - val_loss: 0.5673
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - loss: 0.5526 - val_loss: 0.5535
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.5601 - val_loss: 0.5489
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.5519 - val_loss: 0.5394
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.5238 - val_loss: 0.5346
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 0.5167 - val_loss: 0.5492
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - loss: 0.5534 - val_loss: 0.5297
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 0.5442 - val_loss: 0.5273
Epoch 10/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - loss: 0.5375 - val_loss: 0.5243
Epoch 11/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - loss: 0.5238 - val_loss: 0.5187
Epoch 12/100
108/108 ━━━━━━━━━

[I 2025-04-27 17:18:31,152] Trial 0 finished with value: 0.7947096032202415 and parameters: {'depth': 4, 'n_estimators': 1973, 'learning_rate_weights': 0.18302528605867013, 'learning_rate_index': 0.14970475520083945, 'learning_rate_values': 0.03908905824656489, 'learning_rate_leaf': 0.03908303063201704, 'cosine_decay_steps': 100, 'dropout': 0.5, 'selected_variables': 1.0, 'data_subset_fraction': 0.8, 'focal_loss': True, 'temperature': 0.25}. Best is trial 0 with value: 0.7947096032202415.


Epoch 1/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.7160 - val_loss: 0.5912
Epoch 2/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.6047 - val_loss: 0.5447
Epoch 3/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5585 - val_loss: 0.5509
Epoch 4/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.5885 - val_loss: 0.5461
Epoch 5/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5639 - val_loss: 0.5268
Epoch 6/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5432 - val_loss: 0.5112
Epoch 7/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5403 - val_loss: 0.5523
Epoch 8/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.5741 - val_loss: 0.5435
Epoch 9/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5689 - val_loss: 0.5668
Epoch 10/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.5787 - val_loss: 0.5384
Epoch 11/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.5644 - val_loss: 0.5250
Epoch 12/100
109/109 ━━━━━━━━━

[I 2025-04-27 17:19:16,830] Trial 1 finished with value: 0.7745830937320299 and parameters: {'depth': 3, 'n_estimators': 961, 'learning_rate_weights': 0.09165382463909356, 'learning_rate_index': 0.11407188905583729, 'learning_rate_values': 0.1963154727521141, 'learning_rate_leaf': 0.0499984781613741, 'cosine_decay_steps': 100, 'dropout': 0, 'selected_variables': 0.75, 'data_subset_fraction': 1.0, 'focal_loss': True, 'temperature': 0.25}. Best is trial 0 with value: 0.7947096032202415.


...

Epoch 1/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 175ms/step - loss: 0.5656 - val_loss: 0.4242
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step - loss: 0.4239 - val_loss: 0.3985
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 175ms/step - loss: 0.3969 - val_loss: 0.3883
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step - loss: 0.3821 - val_loss: 0.3820
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 180ms/step - loss: 0.3727 - val_loss: 0.3779
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 176ms/step - loss: 0.3632 - val_loss: 0.3737
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 176ms/step - loss: 0.3563 - val_loss: 0.3726
Epoch 8/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 21s 192ms/step - loss: 0.3498 - val_loss: 0.3714
Epoch 9/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step - loss: 0.3449 - val_loss: 0.3696
Epoch 10/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 178ms/step - loss: 0.3400 - val_loss: 0.3702
Epoch 11/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 19s 180ms/step - loss: 0.3353 - val_loss: 0.3681
Epoch 12

[I 2025-04-27 19:52:11,438] Trial 18 finished with value: 0.8154111558366878 and parameters: {'depth': 6, 'n_estimators': 1288, 'learning_rate_weights': 0.0029555761671426306, 'learning_rate_index': 0.08412792474148581, 'learning_rate_values': 0.04091892611341133, 'learning_rate_leaf': 0.09507735001199624, 'cosine_decay_steps': 0, 'dropout': 0, 'selected_variables': 1.0, 'data_subset_fraction': 0.8, 'focal_loss': False, 'temperature': 0}. Best is trial 16 with value: 0.816561242093157.


Epoch 1/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 67s 610ms/step - loss: 0.6683 - val_loss: 0.5657
Epoch 2/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 66s 603ms/step - loss: 0.5615 - val_loss: 0.5302
Epoch 3/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 585ms/step - loss: 0.5468 - val_loss: 0.5116
Epoch 4/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 63s 580ms/step - loss: 0.5327 - val_loss: 0.4886
Epoch 5/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 66s 603ms/step - loss: 0.5329 - val_loss: 0.5052
Epoch 6/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 588ms/step - loss: 0.5547 - val_loss: 0.5209
Epoch 7/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 592ms/step - loss: 0.5342 - val_loss: 0.5030
Epoch 8/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 583ms/step - loss: 0.5234 - val_loss: 0.5016
Epoch 9/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 586ms/step - loss: 0.5271 - val_loss: 0.6070
Epoch 10/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 586ms/step - loss: 0.5407 - val_loss: 0.4955
Epoch 11/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 64s 586ms/step - loss: 0.5343 - val_loss: 0.5148
Epoch 12

[I 2025-04-27 20:07:18,548] Trial 19 finished with value: 0.7889591719378953 and parameters: {'depth': 7, 'n_estimators': 1235, 'learning_rate_weights': 0.24837124339341893, 'learning_rate_index': 0.12465014895329563, 'learning_rate_values': 0.03752482242956686, 'learning_rate_leaf': 0.16440266648322213, 'cosine_decay_steps': 0, 'dropout': 0, 'selected_variables': 0.5, 'data_subset_fraction': 1.0, 'focal_loss': False, 'temperature': 0.25}. Best is trial 16 with value: 0.816561242093157.


Best CV accuracy: 0.816561242093157
Best params: {'depth': 7, 'n_estimators': 1289, 'learning_rate_weights': 0.012274737330179399, 'learning_rate_index': 0.15721258434952878, 'learning_rate_values': 0.04540455877277873, 'learning_rate_leaf': 0.1180834315364944, 'cosine_decay_steps': 0, 'dropout': 0, 'selected_variables': 0.75, 'data_subset_fraction': 1.0, 'focal_loss': False, 'temperature': 0.25}
Epoch 1/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 69s 625ms/step - loss: 0.6706 - val_loss: 0.4918
Epoch 2/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 69s 636ms/step - loss: 0.5186 - val_loss: 0.4655
Epoch 3/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 71s 650ms/step - loss: 0.4963 - val_loss: 0.4540
Epoch 4/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 68s 620ms/step - loss: 0.4835 - val_loss: 0.4487
Epoch 5/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 68s 622ms/step - loss: 0.4775 - val_loss: 0.4449
Epoch 6/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 67s 617ms/step - loss: 0.4710 - val_loss: 0.4444
Epoch 7/100
109/109 ━━━━━━━━━━━━━━━━━━━━ 68s 621ms/step - lo

## Prediction

In [ ]:
y_pred_raw = final_model.predict(X_valid)
y_pred = np.argmax(y_pred_raw, axis=1)

# Compute accuracy
accuracy = accuracy_score(y_valid, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
test_df = pd.read_csv('data/test_processed.csv')
X_test = test_df.drop(columns=['PassengerId']).values

# Generate predictions with your tuned GRANDE model
preds = final_model.predict(X_test)
# Convert to binary labels
y_test_pred = (preds[:, 1] > 0.5)

# Build submission DataFrame
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_test_pred
})
# Save to CSV
submission.to_csv('submission_grande.csv', index=False)